In [ ]:
!singularity exec \
    --cleanenv \
    --containall \
    -H $PWD:/home \
    --nv \
    stress_slurm/gan.sif \
    /bin/sh -c 'export TF_CPP_MIN_LOG_LEVEL=2 && python3.8 main.py --id 3'
    #/bin/sh -c 'export TF_CPP_MIN_LOG_LEVEL=2 && python3.8 main.py --id 5' #--saving'
    # /bin/sh -c 'export TF_CPP_MIN_LOG_LEVEL=2 && python3.8 noise_attack.py'

***Running scenario 3: 3-TSTR_cGAN
**Starting experiment run (1/3): CNN-LSTM_TSTR_cGAN_syn100_DPCGAN-e-10...
*** Adding real data from: WESAD ***
*** Adding synthetic data from: DPCGAN-e-10 ***
*** Loaded 15 real and 100 synthetic subjects ***

*CI run 1/10...
--CI run duration :0:00:32.737459

*CI run 2/10...
--CI run duration :0:00:28.115035

*CI run 3/10...
--CI run duration :0:00:27.782440

*CI run 4/10...
--CI run duration :0:00:26.295439

*CI run 5/10...
--CI run duration :0:00:24.447303

*CI run 6/10...
--CI run duration :0:00:25.038171

*CI run 7/10...
--CI run duration :0:00:25.241321

*CI run 8/10...
--CI run duration :0:00:26.224663

*CI run 9/10...
--CI run duration :0:00:26.451154

*CI run 10/10...
--CI run duration :0:00:28.162236

***CI results:

         accuracy  precision    recall        f1
S2       0.773529   0.773529  0.773529  0.773529
S3       0.682353   0.682353  0.682353  0.682353
S4       1.000000   1.000000  1.000000  1.000000
S5       0.934286   0.934286  0.

In [14]:
#!cd stress_slurm \
#    && singularity exec gan.sif spython recipe Dockerfile > Singularity \
#    && singularity build gan_.sif Singularity